In [1]:
#reddit account
#username:IES_Python
#email:95346725@fsv.cuni.cz

In [2]:
#import sys
#!{sys.executable} -m pip install praw
#!{sys.executable} -m pip install pandas-datareader
from bs4 import BeautifulSoup
import requests
import time
from datetime import date
import datetime
import pandas as pd
import numpy as np

In [3]:
class Yahoo_Extractor:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self.finalDF = pd.DataFrame({'Date':self.getDates()})
    
    
    def getDates(self):
        dateList = []
        start = date.fromisoformat(self.start_date)
        end = date.fromisoformat(self.end_date)
        numdays = (end - start).days
        for x in range (0, numdays):
            date_name = (end - datetime.timedelta(days = x)).strftime("%b %d, %Y")
            dateList.append(date_name)
        return dateList
    

    def getData(self, acro):
        link = 'https://finance.yahoo.com/quote/' + acro + '/history?p=' + acro
        r = requests.get(link)
        r.encoding = 'utf-8-sig'
        self.soup = BeautifulSoup(r.text, 'html.parser')
        dates = self.getDates()
        stocksDF = pd.DataFrame(index = range(len(dates)), columns = [acro]) #an empty data frame for the stock
        for tr in self.soup.findAll('tr', {'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'}):
            Date = tr.contents[0].text
            try:
                Volume = tr.contents[6].text.replace(",","")
            except IndexError:
                Volume = None
            stock_index = dates.index(Date) #get the index of the traded volume
            stocksDF.loc[stock_index,:] = Volume #replace NaNs with the traded volume for the respective index
        return stocksDF
    
    def joinData(self, acro):
        for a in acro:
            start = time.time()
            stocksDF = self.getData(a)
            self.finalDF = pd.concat([self.finalDF, stocksDF], axis = 1, copy = False)
            end = time.time()
            print('Ellapsed time for stock', a)
            print(end-start)
        self.finalDF = self.finalDF.set_index('Date').dropna(axis = 0, how = 'all')
        #We set the index as 'Date' after the data is concatenated
        return self.finalDF
    

In [4]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
acro = tickers[0]['Symbol'].tolist()             #creating a list of tickers
acro = [a.replace('.', '-') for a in acro] #converting the wikipedia ticker notation into Yahoo Finance ticker notation
#acro = ['MMM', 'ABT', 'BRK-B'] #several acronyms for test



In [5]:
Y = Yahoo_Extractor('2020-05-15','2021-05-15')
outputDF = Y.joinData(acro)

Ellapsed time for stock MMM
2.188479423522949
Ellapsed time for stock ABT
2.0273969173431396
Ellapsed time for stock ABBV
1.8143837451934814
Ellapsed time for stock ABMD
1.5999078750610352
Ellapsed time for stock ACN
1.7596893310546875
Ellapsed time for stock ATVI
2.394426107406616
Ellapsed time for stock ADBE
1.9646995067596436
Ellapsed time for stock AMD
1.585961103439331
Ellapsed time for stock AAP
1.8971798419952393
Ellapsed time for stock AES
2.2305123805999756
Ellapsed time for stock AFL
2.2375166416168213
Ellapsed time for stock A
2.5655901432037354
Ellapsed time for stock APD
2.2633166313171387
Ellapsed time for stock AKAM
1.769437551498413
Ellapsed time for stock ALK
2.0531904697418213
Ellapsed time for stock ALB
2.1824944019317627
Ellapsed time for stock ARE
2.091742753982544
Ellapsed time for stock ALXN
1.7239046096801758
Ellapsed time for stock ALGN
2.3351285457611084
Ellapsed time for stock ALLE
1.89194917678833
Ellapsed time for stock LNT
2.4766881465911865
Ellapsed time 

Ellapsed time for stock EVRG
2.0602004528045654
Ellapsed time for stock ES
2.1445119380950928
Ellapsed time for stock RE
2.2174103260040283
Ellapsed time for stock EXC
2.0784590244293213
Ellapsed time for stock EXPE
2.0395495891571045
Ellapsed time for stock EXPD
1.9946918487548828
Ellapsed time for stock EXR
2.161249876022339
Ellapsed time for stock XOM
1.9506196975708008
Ellapsed time for stock FFIV
1.9007196426391602
Ellapsed time for stock FB
2.3199360370635986
Ellapsed time for stock FAST
2.1513209342956543
Ellapsed time for stock FRT
2.2586331367492676
Ellapsed time for stock FDX
1.9005355834960938
Ellapsed time for stock FIS
1.985992431640625
Ellapsed time for stock FITB
2.193422317504883
Ellapsed time for stock FE
1.646313190460205
Ellapsed time for stock FRC
2.260671615600586
Ellapsed time for stock FISV
1.8138339519500732
Ellapsed time for stock FLT
2.2486157417297363
Ellapsed time for stock FLIR
1.940159797668457
Ellapsed time for stock FMC
2.002190113067627
Ellapsed time fo

Ellapsed time for stock OXY
2.6920900344848633
Ellapsed time for stock ODFL
2.0336732864379883
Ellapsed time for stock OMC
2.02390718460083
Ellapsed time for stock OKE
2.2096052169799805
Ellapsed time for stock ORCL
2.011446237564087
Ellapsed time for stock OTIS
2.125770330429077
Ellapsed time for stock PCAR
2.228956937789917
Ellapsed time for stock PKG
2.553891897201538
Ellapsed time for stock PH
2.549150228500366
Ellapsed time for stock PAYX
2.258826971054077
Ellapsed time for stock PAYC
1.7931280136108398
Ellapsed time for stock PYPL
2.348857879638672
Ellapsed time for stock PENN
1.9839000701904297
Ellapsed time for stock PNR
2.1585652828216553
Ellapsed time for stock PBCT
1.8964462280273438
Ellapsed time for stock PEP
2.302138328552246
Ellapsed time for stock PKI
2.230025053024292
Ellapsed time for stock PRGO
2.1140670776367188
Ellapsed time for stock PFE
2.200915813446045
Ellapsed time for stock PM
2.121495246887207
Ellapsed time for stock PSX
2.047626256942749
Ellapsed time for s

In [6]:
today = date.today()
stringEpoch = today.strftime("%d_%m_%y")
s = ''
nameVars = ["C:\\Users\\hso20\\Python\\Project\\YahooData\\",stringEpoch,".csv"]
outputDF.to_csv(path_or_buf=s.join(nameVars),sep = ',', index=True, encoding='utf-8-sig')